# BaiscTick: Create Cluster: RDB
This notebook will create (start) an RDB cluster on a named database.

In [1]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env2 import *

# Managed KX Database and Cluster names to create
DB_NAME="basictickdb"

SEC_THREADS='8'

BASE_NAME="RDB"
CLUSTER_NAME=f"{BASE_NAME}_{DB_NAME}_20230601"

# Cluster Settings

CODEBASE="basictick"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.2xlarge'}
SAVEDOWN_CONFIG={ 'type': 'SDS01', 'size':400 }
DATABASE_CONFIG=[{ 'databaseName': DB_NAME }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }
INIT_SCRIPT=f'{CODEBASE}/rdbmkdb.q'

CMD_ARGS=[
    {'key': 's', 'value': SEC_THREADS }, 
#    {'key': 'tp', 'value': TP }, 
]

CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'dbname', 'value': DB_NAME}, 
    { 'key': 'codebase', 'value': CODEBASE}, 
    { 'key': 'tphostfile', 'value': TP_HOSTFILE}, 
#    {'key': 'tp', 'value': TP }, 
]

In [3]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## Check Database

In [4]:
note_str = ""

try:
    c_set_list = client.list_kx_changesets(environmentId=ENV_ID, databaseName=DB_NAME)['kxChangesets']
except:
    note_str = "<<Could not get changesets>>"

print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"Changeset ({c['status']}): {c_set_id}: Created: {c['createdTimestamp']}")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: basictickdb, Changesets: 3 
Changeset (COMPLETED): ZsQ3eXNny4UaexiXtmEkZg: Created: 2023-05-30 23:22:00.273000+00:00


changeType,s3Path,dbPath
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.23/,/2023.04.23/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.15/,/2023.04.15/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.14/,/2023.04.14/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.22/,/2023.04.22/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.18/,/2023.04.18/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.20/,/2023.04.20/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.16/,/2023.04.16/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.17/,/2023.04.17/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.21/,/2023.04.21/
PUT,s3://kdb-demo-612841383594/data/hdb/2023.04.19/,/2023.04.19/


Changeset (COMPLETED): QsQ3lbQfnmumBgZOeiPIOw: Created: 2023-05-31 00:18:15.744000+00:00


changeType,s3Path,dbPath
PUT,s3://finspace-landing-us-east-1-itcdoumzc5cixt5vh4t6dp/W9tLqnobb65iKp72sWNF1/0/,/2023.05.31/
PUT,s3://finspace-landing-us-east-1-itcdoumzc5cixt5vh4t6dp/W9tLqnobb65iKp72sWNF1/1/,/


Changeset (COMPLETED): uMQ5qhWLaIRqY97nyAU9cg: Created: 2023-05-31 19:41:15.927000+00:00


changeType,s3Path,dbPath
PUT,s3://finspace-landing-us-east-1-itcdoumzc5cixt5vh4t6dp/57-boC3sloCP4ZeZkw2ql/0/,/2023.05.31/
PUT,s3://finspace-landing-us-east-1-itcdoumzc5cixt5vh4t6dp/57-boC3sloCP4ZeZkw2ql/1/,/


# Deploy code

In [5]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
else:
    cp = f"""
zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
    
# execute the S3 copy
os.system(cp)

updating: basictick/ (stored 0%)
updating: basictick/tp.q (deflated 52%)
updating: basictick/query.q (deflated 19%)
updating: basictick/loadDep.q (deflated 57%)
updating: basictick/funcDownHandle.q (deflated 33%)
updating: basictick/example.schema.q (deflated 12%)
updating: basictick/.ipynb_checkpoints/ (stored 0%)
updating: basictick/.ipynb_checkpoints/query-checkpoint.q (deflated 19%)
updating: basictick/.ipynb_checkpoints/tp-checkpoint.q (deflated 52%)
updating: basictick/.ipynb_checkpoints/tickerplant2-checkpoint (stored 0%)
updating: basictick/.ipynb_checkpoints/tickerplant3-checkpoint (stored 0%)
updating: basictick/tickerplant2 (stored 0%)
updating: basictick/tickerplant3 (stored 0%)
updating: basictick/rdbmkdb.q (deflated 57%)
updating: basictick/hdbmkdb.q (deflated 52%)
updating: basictick/gwmkdbcluster.q (deflated 61%)
updating: basictick/feedmkdb.q (deflated 53%)
updating: basictick/connectmkdb.q (deflated 63%)
updating: basictick/.ipynb_checkpoints/rdbmkdb-checkpoint.q (def

0

## Create Cluster

In [6]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=CLUSTER_NAME,
    clusterType='RDB',
    releaseLabel = '1.0',
    databases=DATABASE_CONFIG,
    capacityConfiguration=CAPACITY_CONFIG,
    savedownStorageConfiguration=SAVEDOWN_CONFIG,
    clusterDescription="Created with create_RDB notebook",
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    commandLineArguments=CMD_ARGS,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [7]:
resp

{'ResponseMetadata': {'RequestId': '238808b8-721f-4f3b-80ef-d1744d8a8ff5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1121',
   'connection': 'keep-alive',
   'date': 'Thu, 01 Jun 2023 14:01:48 GMT',
   'x-amzn-requestid': '238808b8-721f-4f3b-80ef-d1744d8a8ff5',
   'x-amz-apigw-id': 'F16vXE6XoAMFaFw=',
   'x-amzn-trace-id': 'Root=1-6478a4c8-78f207bc0734e7cc1f7f6ae6',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 69baaa5439c683e230d9fcac1c2ffce0.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'XzTL8VNOZIZu774bMpXJ8pDpHkkm0CaDwVW7JVnOcBB-W9BZZ2rsvw=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'RDB_basictickdb_20230601',
 'clusterType': 'RDB',
 'databases': [{'databaseName': 'basictickdb',
   'cacheConfigurations': [],
   'changesetId': 'uMQ5qhWLaIRqY97nyAU9cg'}],
 'clusterDescription': 'Created with create_RDB notebook',
 'capacityConfiguration': {'nodeType': 'kx.s.2xlarge

In [8]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, show_wait=True)
print()
print("** DONE **")

Cluster: RDB_basictickdb_20230601 status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:04:30, waiting 30 sec ...
Cluster: RDB_basictickdb_20230601 status is CREATING, total wait 0:05:0

In [9]:
# Give permissions time to propogate after cluster creation....
time.sleep(20)

In [10]:
conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, userName=KDB_USERNAME, boto_session=session)
print(conn_str)

:tcps://vpce-011b9e5db9449efe9-ssju6ad9.vpce-svc-0c382efe0ad012fe4.us-east-1.vpce.amazonaws.com:443:bob:Host=vpce-011b9e5db9449efe9-ssju6ad9.vpce-svc-0c382efe0ad012fe4.us-east-1.vpce.amazonaws.com&Port=5000&User=bob&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEIf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIGVSJF2sVci%2BFIr%2B6DoUhC%2BJEELqIrHyJ2BKVmFPHnNGAiEAmz2E86TRODJieZ%2FwVZWQ8ltTDpQTaTGK9Pvq8BALPUoqgQMIv%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgw2MTI4NDEzODM1OTQiDI1LqTujfdRMcUm1XSrVAk1TKFDoiFcc4FsQrJ0F9P%2FQ2yB2vpSh%2FNGXmCMrJNEp3m%2FPmBfNNQ4ISOS4E3jVdmHthzNRcdhG2%2BD6%2FzICggCkKRh0aichQWNjgicJpIU6LUmCRpYxd%2FvItJ6yBh6xbxXsu43act%2BjHNPqOAnnOQ6qH9c8UNnn0JhRzo9jR1%2B13cSd%2BJk7L0CfqWILiA7rNKj9XHxmdhvba68fpfCcTEFnPWWEFxhPNLRshKJXWeZkWso%2BdEiwhKzRzpBNy3AhTnyb%2F18fzdB5tKw7TcQY0kqtztJn2%2BvlL%2Fcu8CVkXRfnr2i00bc4Tmg10gXjugQl0xiBpyE3Fo11vWiESSDQgPO%2FlqekJMU%2BDjNhDAZo2p3EIi%2FgpaALncapP3AopRgu57jjQO%2FZBz3X9Tf8WaEha5hZO1qhdlUdUYNd8Iwa8%2Fz263OqPw1m%2

In [11]:
cdf = get_clusters(client, ENV_ID, clusterType="RDB")

display(cdf)

clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
RDB_basictickdb_20230601,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'tphostfile', 'value': 'tickerplant2'}]",Created with create_RDB notebook,2023-06-01 14:15:36.253000+00:00,2023-06-01 14:01:48.077000+00:00,basictickdb,[]
RDB_welcomedb,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_RDB notebook,2023-05-30 23:10:28.477000+00:00,2023-05-30 22:56:52.812000+00:00,welcomedb,[]


In [12]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-06-01 14:16:21.965481
